## Classes (targets)

Class definition based on training examples. The vast majority of training examples are linked with a single target label.
However a small subset (~0.5%), is linked with 2 or more intents. 

Examples:  

- (single-intent) find me a flight from atlanta to baltimore	**flight**
- (multiple-intents) what are the flights and fares from atlanta to philadelphia	**flight+airfare**

Handling Multiple Intents:

**Standalone Class vs. Breakdown**: Despite the presence of the combined labels like flight+airfare, individual appearances of flight and airfare are significantly more common. Therefore, breaking down flight+airfare into separate classes may offer more nuanced classification. This approach ensures that the model can capture the distinctions between individual intents effectively, even if they occasionally appear together.  

**Consideration of Data Distribution**: Given that individual appearances of flight and airfare are more prevalent, prioritizing them as separate classes ensures that the model can learn the specific features and nuances associated with each intent more effectively.


## Training Data Sampling (Few-Shot Classifier)

To create a `few-shot` classifier, we will provide the model with a small number of examples per class, allowing it to gain a better understanding of the context associated with each intent. For example, given 10 unique classes and 1 example per class, we will provide the model with 10 examples, each representing one class.

### Edge Case 1   

Training examples that are linked with more than one class.  
While these data points may offer insights into class correlations, in the context of a few-shot classifier, it's better to avoid using them as training examples due to their infrequency. Based on the training dataset, prompts are correlated with a single label (intent) 99.5% of the time. Examples of multiple target classes (e.g., flight+airfare) are very infrequent; thus, we will refrain from using them as training examples as they could lead the model to create uneven class correlations.

In cases where a class is only present in a multi-labeled example (e.g., `flight+weather`) or is not related to any examples at all, the model will only be provided with the description (name, unique_id) of that class.

Here, if we pass the (**flight+weather**) example to the model, then the very frequent class `flight` might be strongly correlated with the `weather` class. This could result in the model predicting `weather` falsely positive each time a flight-related request appears.


## Evaluation Data Sampling (`zero-shot` and `few-shot`)  

### Edge Case 2    

Assert representative Evaluation (test sampling) regardless of `test_size`.  
Handle edge cases for robust evaluation. 
Class Definition for Evaluation dataset differs from the one in edge-case 1, since this time we want to always include and evaluate multiple intent test cases
for a more robust evaluation.  
Given the aforementioned example the Evaluation dataset is designed to include `flight+weather` as well as individual  `flight` & `weather` class examples.  

**Thus, every evaluation dataset will contain examples for each unique class or class combination found in [test.tsv](../data/atis/test.tsv)**    

If possible, each unique class or class combination will have the same amount of records in the final `test_ds`.  
This strongly depends on `test_size` and the total representation of each unique class or class combination in the [test.tsv](../data/atis/test.tsv).  


1. Identifying Unseen Unique Classes:
    - Compares the unique intents with the intents known to the model (`model_known_targets`).
    - Identifies any intents present in the evaluation dataset but not known to the model as unseen classes (only for individual class, eg. `flight+weather` are not included as unseen).

2. Filtering Unseen Classes:
    - Removes data corresponding to unseen classes from the evaluation dataset.
    - Ensures compatibility between the evaluation dataset and the model's known intents.

3. Adjusting Test Size:
    - Creates a representative evaluation dataset by sampling from the original test dataset.
    - Ensures that each class in the dataset is equally represented in the sampled data (if possible)
    - See `sample_evaluation_data`  in [gpt_intent_classifier](../gpt_intent_classifier.py)


## Training Data and User Query Processing

using nltk and `python string` default methods we:   

- `strip`: user inputs (requests) from leading and trailing spaces  
- `tokenization`: break down string to tokens  
- `stopword removal`: remove words with no strong meaning from the query/ training example  
- `speacial character removal`: remove special characters from query/ training example  
- `reconstruct the query`: recreate the string and pass it to the prompt  

This process shortens input tokens (training examples + text to classify) by 30-40%.  
Too see examples and how this process affects costs visit [prompt_engineering](../notebooks/prompt_engineering.ipynb) & [evaluation_metrics](./evaluation_metrics.ipynb).  


## Addition of new Intents  

To add new intents to the model, modify the [train_ds](../data/atis/train.tsv) to include at least one example of the new class (intent).  
Ideally add also some test records regarding that class in [test.tsv](../data/atis/test.tsv).  

This approach guarantees that `few-shot` classifier will contain at least one training example for the new class to pass to the model.   

In the case of the `zero-shot` classifier the new class `weather` will automatically be assigned a description and label for the model to refer to.  
 
If test records are provided along, then the model's performance will also be evaluated for that class the next time the model loads.  
See [gpt_model_loader](./gpt_model_loader.ipynb) for more information about what happens in the loading process.  

Let's add the new class, `weather` to the model:  

Example:  

add to train set:  

- `weather prediction for Athens tomorrow morning`  ->  weather 

add to test set:  

- `is it raining in London?`    ->  `weather+city`
- `how long will my flight be delayed due to bad weather`   ->  `weather+flight`
- `can airplanes fly safely in stormy weather`  ->  `weather`  

The model now will be able to **recognize** and **evaluate** the new class.  

## Response Validation and Error Handling   

RESPONSE Validation ensures that the received response meets specified criteria or standards.  

- **Purpose**: To verify that the received response aligns with expected formats, structures and content, ensuring data integrity and system reliability.
  
- **Expected Format**: 
  - Data type (e.g., plain text resembling a python list of 3 integers)  
  - Integers correspond to actual intent labels  


**Evaluation Process**:

The service checks if a given string looks like a list of 3 numbers. It ensures that:  
- The string is properly formatted, resembling a list of 3 integers.  
- Integers correspond to intent_labels and are not new.    
- It removes any unnecessary spaces before and after the string.  
- Returns `True` if the string follows the expected format, and `False` otherwise. 
sing fields, unexpected data types, or invalid values.  
  
- **Outcome**: Successful RESPONSE Validation confirms the reliability and trustworthiness of the received data, enabling further processing or action with confidence. 
Failed validation may indicate issues such as data corruption, communication errors, or non-compliance with specifications.   

See [server_launch_examples](../server_launch_examples.ipynb) (`Test Response Validator` section) to see examples of invalid model outputs, as well as a few slighly malformed outputs that the model handles effectively.   
  
## Error handling  

Invalid responses for requests to the `/intent` endpoint will result in:  

### 500 INTERNAL ERROR  

```json
{"label": "INTERNAL_ERROR", "message": "Invalid Model Response"}
```

Invalid responses during the evaluation process are logged and counted without throwing an error.  

See some examples of invalid responses at [malformed_responses.txt](../model_evaluation/malformed_responses.txt). 
